In [35]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# For preprocessing and splitting data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# For models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# For evaluating models
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, mean_squared_error, r2_score

# For deep learning (TensorFlow/Keras)
import tensorflow as tf
from tensorflow import keras

# xgboost
import xgboost as xgb

In [29]:
rankings = pd.read_csv('LeagueRanking.csv')
rankings

,Position,Region,Player,Total,International,Unnamed: 5,Regional - East,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 56,Mid-Season,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65
0,NaN,NaN,NaN,Points,1st,2nd,1st,2nd,MVP,All-Pro,...,Points,First MSI,Second MSI,First MSC,Second MSC,1° AS 2014,2° AS 2014,Final MVP,Semi MVP,Points
1,Mid,LCK,Faker,1121,8,4,9,6,1,14,...,370,2,2,0,0,1,0,1,3,130
2,Mid,LPL,Knight,593,2,2,5,3,3,11,...,59,1,1,1,0,0,0,1,1,74
3,Mid,LCK,Chovy,572,1,0,4,6,3,12,...,28,1,0,0,0,0,0,0,1,36
4,Jungle,LCK,Peanut,542,1,2,7,3,1,12,...,47,1,1,0,0,0,0,0,0,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466,Top,LCS,KiWiKiD,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
467,Mid,LCS,Keane,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
468,Supp,LCS,Hakuho,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
469,Supp,LCS,Elementz,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
# Select the desired columns if they exist
rank = rankings[['Player', 'Total']]
rank = rank.dropna()
rank

,Player,Total
1,Faker,1121
2,Knight,593
3,Chovy,572
4,Peanut,542
5,Keria,500
...,...,...
466,KiWiKiD,1
467,Keane,1
468,Hakuho,1
469,Elementz,1


In [40]:
# Path to the folder containing your CSV files
folder_path = 'OraclesElixir'

# List all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

combined_df = pd.DataFrame()

# Iterate over each CSV file
for file in csv_files:
    # Read the file in chunks (adjust chunk size as needed)
    for chunk in pd.read_csv(os.path.join(folder_path, file), chunksize=100000):
        combined_df = pd.concat([combined_df, chunk], ignore_index=True)

combined_df

/var/folders/0h/ch124vqs7dj_96gjw7qq3ksh0000gn/T/ipykernel_86315/4051504149.py:12: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(os.path.join(folder_path, file), chunksize=100000):
/var/folders/0h/ch124vqs7dj_96gjw7qq3ksh0000gn/T/ipykernel_86315/4051504149.py:12: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(os.path.join(folder_path, file), chunksize=100000):
/var/folders/0h/ch124vqs7dj_96gjw7qq3ksh0000gn/T/ipykernel_86315/4051504149.py:12: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(os.path.join(folder_path, file), chunksize=100000):
/var/folders/0h/ch124vqs7dj_96gjw7qq3ksh0000gn/T/ipykernel_86315/4051504149.py:12: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(os.p

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,opp_csat25,golddiffat25,xpdiffat25,csdiffat25,killsat25,assistsat25,deathsat25,opp_killsat25,opp_assistsat25,opp_deathsat25
0,ESPORTSTMNT01/1030526,complete,https://matchhistory.br.leagueoflegends.com/pt...,CBLOL,2019,Split 1,0,2019-01-12 14:56:22,1.0,9.01,...,237.0,-939.0,-1230.0,-15.0,2.0,1.0,2.0,4.0,0.0,2.0
1,ESPORTSTMNT01/1030526,complete,https://matchhistory.br.leagueoflegends.com/pt...,CBLOL,2019,Split 1,0,2019-01-12 14:56:22,1.0,9.01,...,137.0,1763.0,1851.0,24.0,2.0,7.0,2.0,0.0,1.0,1.0
2,ESPORTSTMNT01/1030526,complete,https://matchhistory.br.leagueoflegends.com/pt...,CBLOL,2019,Split 1,0,2019-01-12 14:56:22,1.0,9.01,...,201.0,1923.0,934.0,36.0,2.0,2.0,0.0,0.0,2.0,1.0
3,ESPORTSTMNT01/1030526,complete,https://matchhistory.br.leagueoflegends.com/pt...,CBLOL,2019,Split 1,0,2019-01-12 14:56:22,1.0,9.01,...,234.0,1886.0,927.0,3.0,4.0,5.0,1.0,0.0,2.0,4.0
4,ESPORTSTMNT01/1030526,complete,https://matchhistory.br.leagueoflegends.com/pt...,CBLOL,2019,Split 1,0,2019-01-12 14:56:22,1.0,9.01,...,18.0,2378.0,1226.0,25.0,1.0,9.0,1.0,1.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992215,2877-3122,partial,https://lpl.qq.com/es/stats.shtml?bmid=2877,DCup,2017,NaN,0,2017-12-31 11:43:15,2.0,7.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
992216,2877-3122,partial,https://lpl.qq.com/es/stats.shtml?bmid=2877,DCup,2017,NaN,0,2017-12-31 11:43:15,2.0,7.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
992217,2877-3122,partial,https://lpl.qq.com/es/stats.shtml?bmid=2877,DCup,2017,NaN,0,2017-12-31 11:43:15,2.0,7.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
992218,2877-3122,partial,https://lpl.qq.com/es/stats.shtml?bmid=2877,DCup,2017,NaN,0,2017-12-31 11:43:15,2.0,7.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
combined_df = combined_df[combined_df['datacompleteness'] == 'complete']
combined_df

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,opp_csat25,golddiffat25,xpdiffat25,csdiffat25,killsat25,assistsat25,deathsat25,opp_killsat25,opp_assistsat25,opp_deathsat25
0,ESPORTSTMNT01/1030526,complete,https://matchhistory.br.leagueoflegends.com/pt...,CBLOL,2019,Split 1,0,2019-01-12 14:56:22,1.0,9.01,...,237.0,-939.0,-1230.0,-15.0,2.0,1.0,2.0,4.0,0.0,2.0
1,ESPORTSTMNT01/1030526,complete,https://matchhistory.br.leagueoflegends.com/pt...,CBLOL,2019,Split 1,0,2019-01-12 14:56:22,1.0,9.01,...,137.0,1763.0,1851.0,24.0,2.0,7.0,2.0,0.0,1.0,1.0
2,ESPORTSTMNT01/1030526,complete,https://matchhistory.br.leagueoflegends.com/pt...,CBLOL,2019,Split 1,0,2019-01-12 14:56:22,1.0,9.01,...,201.0,1923.0,934.0,36.0,2.0,2.0,0.0,0.0,2.0,1.0
3,ESPORTSTMNT01/1030526,complete,https://matchhistory.br.leagueoflegends.com/pt...,CBLOL,2019,Split 1,0,2019-01-12 14:56:22,1.0,9.01,...,234.0,1886.0,927.0,3.0,4.0,5.0,1.0,0.0,2.0,4.0
4,ESPORTSTMNT01/1030526,complete,https://matchhistory.br.leagueoflegends.com/pt...,CBLOL,2019,Split 1,0,2019-01-12 14:56:22,1.0,9.01,...,18.0,2378.0,1226.0,25.0,1.0,9.0,1.0,1.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991735,ESPORTSTMNT06/500077,complete,https://matchhistory.na.leagueoflegends.com/en...,CK,2018,Spring,0,2017-12-17 12:01:14,1.0,7.23,...,209.0,3566.0,2012.0,53.0,5.0,3.0,3.0,0.0,3.0,2.0
991736,ESPORTSTMNT06/500077,complete,https://matchhistory.na.leagueoflegends.com/en...,CK,2018,Spring,0,2017-12-17 12:01:14,1.0,7.23,...,222.0,1563.0,840.0,49.0,4.0,3.0,1.0,2.0,3.0,3.0
991737,ESPORTSTMNT06/500077,complete,https://matchhistory.na.leagueoflegends.com/en...,CK,2018,Spring,0,2017-12-17 12:01:14,1.0,7.23,...,24.0,443.0,-378.0,24.0,0.0,6.0,1.0,1.0,4.0,3.0
991738,ESPORTSTMNT06/500077,complete,https://matchhistory.na.leagueoflegends.com/en...,CK,2018,Spring,0,2017-12-17 12:01:14,1.0,7.23,...,928.0,-8560.0,-3787.0,-211.0,8.0,14.0,13.0,13.0,20.0,8.0


In [45]:
combined_df = combined_df.sort_values(by='date').reset_index()
combined_df

,index,gameid,datacompleteness,url,league,year,split,playoffs,date,game,...,opp_csat25,golddiffat25,xpdiffat25,csdiffat25,killsat25,assistsat25,deathsat25,opp_killsat25,opp_assistsat25,opp_deathsat25
0,862222,TRLH3/33,complete,http://matchhistory.na.leagueoflegends.com/en/...,EU LCS,2014,Spring,0,2014-01-14 17:52:02,1.0,...,760.0,4854.0,4919.0,-21.0,10.0,23.0,4.0,4.0,6.0,10.0
1,862214,TRLH3/33,complete,http://matchhistory.na.leagueoflegends.com/en/...,EU LCS,2014,Spring,0,2014-01-14 17:52:02,1.0,...,225.0,621.0,733.0,8.0,1.0,5.0,1.0,1.0,2.0,0.0
2,862212,TRLH3/33,complete,http://matchhistory.na.leagueoflegends.com/en/...,EU LCS,2014,Spring,0,2014-01-14 17:52:02,1.0,...,206.0,76.0,-512.0,-18.0,3.0,4.0,0.0,1.0,2.0,2.0
3,862213,TRLH3/33,complete,http://matchhistory.na.leagueoflegends.com/en/...,EU LCS,2014,Spring,0,2014-01-14 17:52:02,1.0,...,140.0,-888.0,351.0,-42.0,0.0,5.0,3.0,2.0,1.0,1.0
4,862216,TRLH3/33,complete,http://matchhistory.na.leagueoflegends.com/en/...,EU LCS,2014,Spring,0,2014-01-14 17:52:02,1.0,...,28.0,1780.0,2397.0,-19.0,0.0,7.0,0.0,0.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
872023,292715,LOLTMNT02_181861,complete,NaN,HW,2024,NaN,0,2024-11-03 21:08:36,5.0,...,870.0,2295.0,7637.0,-8.0,9.0,23.0,4.0,4.0,9.0,9.0
872024,292706,LOLTMNT02_181861,complete,NaN,HW,2024,NaN,0,2024-11-03 21:08:36,5.0,...,214.0,-716.0,-2609.0,-14.0,1.0,2.0,3.0,1.0,6.0,0.0
872025,292711,LOLTMNT02_181861,complete,NaN,HW,2024,NaN,0,2024-11-03 21:08:36,5.0,...,200.0,716.0,2609.0,14.0,1.0,6.0,0.0,1.0,2.0,3.0
872026,292712,LOLTMNT02_181861,complete,NaN,HW,2024,NaN,0,2024-11-03 21:08:36,5.0,...,264.0,-445.0,1274.0,-28.0,1.0,5.0,0.0,1.0,2.0,0.0


In [44]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 872028 entries, 862222 to 292704
Columns: 161 entries, gameid to opp_deathsat25
dtypes: float64(128), int64(10), object(23)
memory usage: 1.1+ GB


In [46]:
combined_df.to_csv('OEFULL.csv', index=False)